In [1]:
# IMPORT
import os
import tweepy
import time 
import urllib.request, urllib.error
import pandas as pd

# 画像の保存先
#保存先のパス名を「＊＊＊＊＊」に入力
IMG_DIR = './リコリコ/'

# 環境変数
CONSUMER_KEY        = "F15H5jdEJvYRR6laSwJ2AOxPi"
CONSUMER_SECRET     = "ealZBAVgFAapP3CDay1bavc2od9S5uUsDQjHbBiUUBeHYxlce7"
ACCESS_TOKEN_KEY    = "1373901087070060547-5tP5TZOBuePU2cEjO1So13ue6FF0Rd"
ACCESS_TOKEN_SECRET = "nTCjHaTrYCK4gQ8IEvmMcCiaaQVY5ucy2yFfXp9OzjYOp"

# 検索キーワード
#好きなキーワードを「＊＊＊＊＊」に入力
TARGET = '#リコリコ OR 千束 OR たきな OR ちさたき OR たきちさ min_faves:400 min_retweets:200'

labels=[
        'ツイートID',
        'ツイート時刻',
        'ツイート本文',
        'いいね数',
        'リツイート数',
        'ユーザーID',
        'ユーザー名',
        'アカウント名',
        '自己紹介文',
        'フォロー数',
        'フォロワー数',
        'アカウント作成日時',
        '自分のフォロー状況',
        'アイコン画像URL',
        'ヘッダー画像URL',
        'WEBサイト',
        '画像URL1',
        '画像URL2',
        '画像URL3',
        '画像URL4',
        ]

# 検索オプション
SEARCH_PAGES_NUMBER = 10000 # 読み込むページ数
PER_PAGE_NUMBER = 100 # ページごとに返されるツイートの数（最大100）

In [4]:
class imageDownloader(object):
    def __init__(self):
        """初期設定
        """
        super(imageDownloader, self).__init__()
        self.set_api()

    def run(self):
        """実行
            1. twitterページを指定数取得
            2. ページ内のツイートのうち、キーワードがあるtweetのみ取得
            3. 画像URLを取得
            4. ダウンロード実行
        """
        self.max_id = None # ページを跨ぐ検索対象IDの初期化
        for page in range(SEARCH_PAGES_NUMBER):
            ret_url_list = self.search(TARGET, PER_PAGE_NUMBER)
            for url in ret_url_list:
                print('OK ' + url)
                self.download(url)
            time.sleep(0.5) # TimeOut防止

    def set_api(self):
        """apiの設定
        """
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth)

    def search(self, target, rpp):
        """twitterで検索実行
        """
        # 検索結果
        ret_url_list = []

        try:
            # 検索実行
            if self.max_id:
                # q: クエリ文字列, rpp: ツイート数, max_id: より小さい（古い）IDを持つステータスのみを返す
                res_search = self.api.search_tweets(q=target, lang='ja', count=rpp, max_id=self.max_id, tweet_mode = 'extended')
            else:
                res_search = self.api.search_tweets(q=target, lang='ja', count=rpp, tweet_mode = 'extended')
            # 結果を保存
            save_tweet_data(res_search)
            for result in res_search:
                if 'media' not in result.entities: continue
                for media in result.extended_entities['media']:
                    url = media['media_url_https']
                    if url not in ret_url_list: ret_url_list.append(url)
            # 検索済みidの更新し、より古いツイートを検索させる
            self.max_id = result.id
            # 検索結果の返却
            return ret_url_list
        except Exception as e:
            self.error_catch(e)

    def download(self, url):
        """画像のダウンロード
        """
        url_orig = '%s?format=jpg&name=4096x4096' % url
        path = IMG_DIR + url.split('/')[-1]
        try:
            response = urllib.request.urlopen(url=url_orig)
            with open(path, "wb") as f:
                f.write(response.read())
        except Exception as e:
            self.error_catch(e)

    def error_catch(self, error):
        """エラー処理
        """
        print("NG ", error)

In [5]:
def save_tweet_data(res_search):
    tw_data = []
    
    for result in res_search:
        if 'media' not in result.entities: continue
        tw_data.append([
                result.id,
                result.created_at.strftime('%Y-%m-%d %H:%M:%S'),
                result.full_text,
                result.favorite_count, 
                result.retweet_count, 
                result.user.id, 
                result.user.screen_name,
                result.user.name,
                result.user.description,
                result.user.friends_count,
                result.user.followers_count,
                result.user.created_at.strftime('%Y-%m-%d %H:%M:%S'),
                result.user.following,
                result.user.profile_image_url,
                result.user.profile_background_image_url,
                result.user.url
                               ])
        media_urls = [media['media_url_https'] for media in result.extended_entities["media"]]
        tw_data[-1] += media_urls
        while len(tw_data[-1]) < 20:
            tw_data[-1].append(None)
    
    df = pd.DataFrame(tw_data,columns=labels)
    df.to_csv("./tweet.csv", mode='a', header=False, index=False)

In [6]:
"""メイン処理
"""
try:
    downloader = imageDownloader()
    downloader.run()
except KeyboardInterrupt:
    # Ctrl-Cで終了
    pass


OK https://pbs.twimg.com/media/FdWV4_TaIAY-bqH.jpg
OK https://pbs.twimg.com/media/FdUf9uqagAAEihf.png
OK https://pbs.twimg.com/media/FdWUMSpaUAAmANn.jpg
OK https://pbs.twimg.com/media/FdWUHRkXgAErzj7.jpg
OK https://pbs.twimg.com/media/FdNFfXuacAEgslj.jpg
OK https://pbs.twimg.com/media/FdWGaCjUoAE0W5o.jpg
OK https://pbs.twimg.com/media/FdWErXQUcAIki3m.jpg
OK https://pbs.twimg.com/media/FdV2XSYakAYj7fg.jpg
OK https://pbs.twimg.com/media/FdV2XSaaUAAuCCk.jpg
OK https://pbs.twimg.com/media/FdV2XSeaEAIVogS.jpg
OK https://pbs.twimg.com/media/FdV0aY1agAEffAS.jpg
OK https://pbs.twimg.com/media/FdVpcUWaAAEh5Wg.jpg
OK https://pbs.twimg.com/media/FdVoWSWVEAEVbHD.jpg
OK https://pbs.twimg.com/media/FdVmoHMakAcwwUO.jpg
OK https://pbs.twimg.com/media/FdVluIvaAAIhOUc.jpg
OK https://pbs.twimg.com/media/FdVeBHTaUAAjLUC.jpg
OK https://pbs.twimg.com/media/FdVZ8vDacAAYbgo.jpg
OK https://pbs.twimg.com/media/FdTtc0tUYAIuwJX.jpg
OK https://pbs.twimg.com/media/FdVVK24VQAA_VZN.jpg
OK https://pbs.twimg.com/media/

TypeError: 'NoneType' object is not iterable